# 1. Data Loading

1. DEMOGRAPHIC_selected_cols parquet file contains all rows with selected columns (voter id, city, county, ethnicity, age, gender, education, income, donation and parties description). See Reduce_to_parquet.ipynb 
2. VOTEHISTORY_selected_cols parquet file contains all rows with selected columns (General and Local_or_Municipal). See Reduce_to_parquet.ipynb 
3. GE_LM_dates_per_city parquet file contains four most recent General election and Local_or_Municipal for each of the selected city. See Find_recent_election_dates.ipynb and ca_similarity_search.ipynb

Some observations
- We choose the 5 non-RCV cities with highest cosine similary score compared to the 7 RCV cities in CA
- There were 33 distinct cities among those 35 cities
- There are 66 non-registered voters among 21.7 million voters
- There are total of 3.9 million voters in the sampled cities
- City 'El Paso de Robles' didn't match in demographic data


In [1]:
import pandas as pd
import janitor
import gc
import time
start_time = time.time()

In [2]:
# ------ California -------

# change the filepath as required, we have selected the folder with the latest date
filepath = '../data/VM2--CA--2022-04-25/'

DEMO_filename = 'VM2--CA--2022-04-25-DEMOGRAPHIC_selected_cols.parquet'

VOTE_filename = 'VM2--CA--2022-04-25-VOTEHISTORY_selected_cols.parquet'

elec_dates_filename = 'GE_LM_dates_per_city.parquet'

In [3]:
def combine_cities_list(RCV_list, NonRCV_list):

    print("total number of cities:", len(RCV_list))

    print("number of distinct cities:", len(set(NonRCV_list)))

    print("name of cities that were duplicated:", set([x for x in NonRCV_list if NonRCV_list.count(x) > 1]))

    combined_cityName = RCV_list+list(set(NonRCV_list))
    print("number of distinct RCV and sampled nonRCV cities:", len(combined_cityName))
    return combined_cityName


In [4]:
# 1. List of RCV and non-RCV cities 

# ------ California -------

RCV_cities_CA = ['San Francisco',
 'Oakland',
 'Berkeley',
 'San Leandro',
 'Palm Desert',
 'Eureka',
 'Albany']

sampled_nonRCV_cities_CA = ['Fresno',
 'San Diego',
 'Sacramento',
 'Riverside',
 'San Jose',
 'Santa Ana',
 'Anaheim',
 'Santa Rosa',
 'Merced',
 'Santa Clarita',
 'Alhambra',
 'Davis',
 'Montebello',
 'Burbank',
 'Huntington Park',
 'Bellflower',
 'Watsonville',
 'Gilroy',
 'Whittier',
 'Lynwood',
 'Lakewood',
 'Pico Rivera',
 'Lake Forest',
 'Livermore',
 'Chino Hills',
 'Paramount',
 'El Paso de Robles',
 'Pico Rivera',
 'Buena Park',
 'Whittier',
 'Calabasas',
 'Carpinteria',
 'Morro Bay',
 'San Carlos',
 'Solvang']

combined_sampled_cityName = combine_cities_list(RCV_list= RCV_cities_CA, NonRCV_list = sampled_nonRCV_cities_CA)
# ---------------------


total number of cities: 7
number of distinct cities: 33
name of cities that were duplicated: {'Whittier', 'Pico Rivera'}
number of distinct RCV and sampled nonRCV cities: 40


# 1.1 Demographic Data

1. Select only the columns required: city name ('Residence_Addresses_City'), unique voter id ('LALVOTERID'), voter's ethnicity ('EthnicGroups_EthnicGroup1Desc'), date when voter was registered ('Voters_OfficialRegDate'), voter's gender, date of birth, plus additionsl columns
2. Keep only the cities that were identified as being similar to RCV cities in CA (See ca_similarity_search.ipynb for reference) 
3. Keep only rows EthnicGroups_EthnicGroup1Desc == “European”,  “Likely African-American”,“Hispanic and Portuguese” and “East and South Asian” 
4. Keep only registered voters identified in 'Voters_OfficialRegDate'


In [5]:
def read_DEMOGRAPHIC():
    df_demographic = pd.read_parquet(f'{filepath}{DEMO_filename}')
    print("Total number of unique cities:", df_demographic.Residence_Addresses_City.nunique())
    print("Total number of unique voters:", df_demographic.LALVOTERID.nunique())
    print("Count of non-registered voters:", len(df_demographic[df_demographic['Voters_OfficialRegDate'].isnull()]))
    
    print("Number of expected cities:", len(combined_sampled_cityName))
    missing_cities = [city for city in combined_sampled_cityName if city not in df_demographic['Residence_Addresses_City'].unique()]
    if len(missing_cities) > 0:
        print("number of cities not found in demographic data:", len(missing_cities))
        print(missing_cities)
        
    return df_demographic
        
state_demographic = read_DEMOGRAPHIC()

Total number of unique cities: 1533
Total number of unique voters: 21711617
Count of non-registered voters: 66
Number of expected cities: 40
number of cities not found in demographic data: 1
['El Paso de Robles']


In [6]:
state_demographic.head(5)

,LALVOTERID,Residence_Addresses_City,Voters_Gender,Voters_Age,Parties_Description,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate,County,CommercialData_Education,CommercialData_EstimatedHHIncomeAmount,FECDonors_NumberOfDonations,FECDonors_TotalDonationsAmount
0,LALCA453164106,Oakland,F,29,Democratic,Other,06/18/2021,ALAMEDA,None,None,None,None
1,LALCA453008306,Oakland,F,26,Non-Partisan,Likely African-American,04/01/2021,ALAMEDA,None,None,None,None
2,LALCA22129469,Oakland,F,47,Democratic,European,11/16/2021,ALAMEDA,HS Diploma - Extremely Likely,None,None,None
3,LALCA549803906,Oakland,M,60,Democratic,Other,02/07/2022,ALAMEDA,None,None,None,None
4,LALCA24729024,San Leandro,F,56,Democratic,European,02/28/2016,ALAMEDA,HS Diploma - Extremely Likely,None,None,None


In [7]:
# ----- California ----- 
combined_sampled_cityName = list(map(lambda x: x.replace('El Paso de Robles', 'Paso Robles'), combined_sampled_cityName))
print("number of expected cities:", len(combined_sampled_cityName))
# ----------------------


number of expected cities: 40


In [8]:
# 2. filter DEMOGRAPHIC data based on the list of cities, ethnicities and registered voters

selected_ethnicities = ['European', 'Likely African-American','Hispanic and Portuguese', 'East and South Asian']

def filter_demo(df, list_cityNames):
    filtered_df = df[df['Residence_Addresses_City'].isin(list_cityNames) &
            df['EthnicGroups_EthnicGroup1Desc'].isin(selected_ethnicities) &
            df['Voters_OfficialRegDate'].notnull()]
    #[['LALVOTERID', 'Residence_Addresses_City']]
    
    print(filtered_df.shape)
    print("number of unique cities:", filtered_df.Residence_Addresses_City.nunique())
    
    return filtered_df

state_demographic_subset = filter_demo(df = state_demographic, list_cityNames = combined_sampled_cityName)
state_demographic_subset.head()

(3944492, 12)
number of unique cities: 40


,LALVOTERID,Residence_Addresses_City,Voters_Gender,Voters_Age,Parties_Description,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate,County,CommercialData_Education,CommercialData_EstimatedHHIncomeAmount,FECDonors_NumberOfDonations,FECDonors_TotalDonationsAmount
1,LALCA453008306,Oakland,F,26,Non-Partisan,Likely African-American,04/01/2021,ALAMEDA,None,None,None,None
2,LALCA22129469,Oakland,F,47,Democratic,European,11/16/2021,ALAMEDA,HS Diploma - Extremely Likely,None,None,None
4,LALCA24729024,San Leandro,F,56,Democratic,European,02/28/2016,ALAMEDA,HS Diploma - Extremely Likely,None,None,None
6,LALCA22466723,Livermore,F,38,Republican,European,11/01/2021,ALAMEDA,None,None,None,None
7,LALCA22466636,Livermore,M,63,Democratic,European,12/07/2021,ALAMEDA,None,None,None,None


In [9]:
gc.collect()

20

In [10]:
del state_demographic
gc.collect()

0

# 1.2 Vote History

1. Select only the columns that are 4 most recent General elections and 4 most recent Local_or_Municipal elections and EthnicGroups_EthnicGroup1Desc
2. Load Vote History 
3. Merge Vote History with the sampled Demographic Data 


## 1. Get four most recent election dates

In [11]:
# load the list of election dates for each city
GE_LM_dates_list = pd.read_parquet(f'{filepath}{elec_dates_filename}')
GE_LM_dates_list

,city,GE_dates,LM_dates
0,Oakland,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2020_08_03, Local_or_Munic..."
1,San Leandro,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2020_08_03, Local_or_Munic..."
2,Livermore,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2020_08_03, Local_or_Munic..."
3,Berkeley,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2021_06_01, Local_or_Munic..."
4,Albany,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2021_03_02, Local_or_Munic..."
5,San Francisco,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2021_07_20, Local_or_Munic..."
6,San Diego,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2021_06_08, Local_or_Munic..."
7,San Jose,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2021_06_08, Local_or_Munic..."
8,Fresno,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2021_06_01, Local_or_Munic..."
9,Eureka,"[General_2020_11_03, General_2018_11_06, Gener...","[Local_or_Municipal_2021_06_08, Local_or_Munic..."


In [12]:
e_dates = set()
for v in GE_LM_dates_list['GE_dates']:
    for vv in v:
        e_dates.add(vv)
for v in GE_LM_dates_list['LM_dates'] :
    for vv in v:
        e_dates.add(vv)

In [13]:
list(e_dates)

['Local_or_Municipal_2021_06_08',
 'Local_or_Municipal_2021_04_20',
 'Local_or_Municipal_2021_05_11',
 'Local_or_Municipal_2019_08_13',
 'Local_or_Municipal_2021_03_02',
 'Local_or_Municipal_2017_03_07',
 'General_2016_11_08',
 'Local_or_Municipal_2017_05_09',
 'Local_or_Municipal_2018_04_10',
 'Local_or_Municipal_2016_04_12',
 'Local_or_Municipal_2020_04_14',
 'General_2014_11_04',
 'Local_or_Municipal_2019_04_16',
 'Local_or_Municipal_2019_03_05',
 'Local_or_Municipal_2019_08_27',
 'Local_or_Municipal_2021_07_20',
 'General_2020_11_03',
 'Local_or_Municipal_2017_06_06',
 'Local_or_Municipal_2017_08_29',
 'Local_or_Municipal_2021_06_01',
 'Local_or_Municipal_2020_08_03',
 'Local_or_Municipal_2019_06_04',
 'Local_or_Municipal_2017_05_02',
 'Local_or_Municipal_2018_07_24',
 'General_2018_11_06']

In [14]:
# need in order to filter out rows after aggregation
def get_correct_dates(list_like_df):
    print("Shape before reshaping:",list_like_df.shape)
    list_like_df = list_like_df.explode(['GE_dates', 'LM_dates']).melt(id_vars=["city"], 
                                                                       var_name="Date", 
                                                                       value_name="Value")
    list_like_df = list_like_df.drop(columns = 'Date')
    list_like_df.columns = ['Residence_Addresses_City', 'elec_type_date']                          
    list_like_df['elec_date'] = list_like_df['elec_type_date'].str[-10:]
    list_like_df['elec_year'] = list_like_df['elec_type_date'].str[-10:-6]
    list_like_df['elec_type'] = list_like_df['elec_type_date'].str[:-11]                    
    list_like_df = list_like_df.drop(columns = 'elec_type_date')
    print("Shape after reshaping:",list_like_df.shape)
    return list_like_df

GE_LM_dates_df = get_correct_dates(GE_LM_dates_list)
GE_LM_dates_df.head()

Shape before reshaping: (40, 3)
Shape after reshaping: (320, 4)


,Residence_Addresses_City,elec_date,elec_year,elec_type
0,Oakland,2020_11_03,2020,General
1,Oakland,2018_11_06,2018,General
2,Oakland,2016_11_08,2016,General
3,Oakland,2014_11_04,2014,General
4,San Leandro,2020_11_03,2020,General


## 2. load the VOTE HISTORY data for selected election dates only

In [15]:
needed_variables = ['LALVOTERID'] + list(e_dates)

state_voterhistory_4_dates = pd.read_parquet(f'{filepath}{VOTE_filename}',
                                             columns=needed_variables)
                                
state_voterhistory_4_dates.head(5)

,LALVOTERID,Local_or_Municipal_2021_06_08,Local_or_Municipal_2021_04_20,Local_or_Municipal_2021_05_11,Local_or_Municipal_2019_08_13,Local_or_Municipal_2021_03_02,Local_or_Municipal_2017_03_07,General_2016_11_08,Local_or_Municipal_2017_05_09,Local_or_Municipal_2018_04_10,...,Local_or_Municipal_2021_07_20,General_2020_11_03,Local_or_Municipal_2017_06_06,Local_or_Municipal_2017_08_29,Local_or_Municipal_2021_06_01,Local_or_Municipal_2020_08_03,Local_or_Municipal_2019_06_04,Local_or_Municipal_2017_05_02,Local_or_Municipal_2018_07_24,General_2018_11_06
0,LALCA453164106,None,None,None,None,None,None,Y,None,None,...,None,Y,None,None,None,None,None,None,None,Y
1,LALCA453008306,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,Y
2,LALCA22129469,None,None,None,None,None,None,Y,None,None,...,None,Y,None,None,None,None,None,None,None,Y
3,LALCA549803906,None,None,None,None,None,None,None,None,None,...,None,Y,None,None,None,None,None,None,None,None
4,LALCA24729024,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## 3. Merge Vote History and Demographic Data

In [16]:
merged_file = pd.merge(state_voterhistory_4_dates, state_demographic_subset,
                       how='inner', left_on='LALVOTERID', right_on='LALVOTERID')

print(merged_file.shape)

print("number of unique cities:", merged_file.Residence_Addresses_City.nunique())

merged_file.head(5)

(3944492, 37)
number of unique cities: 40


,LALVOTERID,Local_or_Municipal_2021_06_08,Local_or_Municipal_2021_04_20,Local_or_Municipal_2021_05_11,Local_or_Municipal_2019_08_13,Local_or_Municipal_2021_03_02,Local_or_Municipal_2017_03_07,General_2016_11_08,Local_or_Municipal_2017_05_09,Local_or_Municipal_2018_04_10,...,Voters_Gender,Voters_Age,Parties_Description,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate,County,CommercialData_Education,CommercialData_EstimatedHHIncomeAmount,FECDonors_NumberOfDonations,FECDonors_TotalDonationsAmount
0,LALCA453008306,None,None,None,None,None,None,None,None,None,...,F,26,Non-Partisan,Likely African-American,04/01/2021,ALAMEDA,None,None,None,None
1,LALCA22129469,None,None,None,None,None,None,Y,None,None,...,F,47,Democratic,European,11/16/2021,ALAMEDA,HS Diploma - Extremely Likely,None,None,None
2,LALCA24729024,None,None,None,None,None,None,None,None,None,...,F,56,Democratic,European,02/28/2016,ALAMEDA,HS Diploma - Extremely Likely,None,None,None
3,LALCA22466723,None,None,None,None,None,None,None,None,None,...,F,38,Republican,European,11/01/2021,ALAMEDA,None,None,None,None
4,LALCA22466636,None,None,None,None,None,None,Y,None,None,...,M,63,Democratic,European,12/07/2021,ALAMEDA,None,None,None,None


In [17]:
merged_file.to_parquet(f'{filepath}VM2--CA--2022-04-25-MERGED.parquet')

# 3.1. Calculate voter turnout per ethnicity

In [18]:
# import pandas as pd
# merged_file = pd.read_parquet(f'{file}VM2--CA--2022-04-25-MERGED.parquet')

In [19]:
def replace_ethnicities(df):
    df = df.replace('East and South Asian', 'asian')
    df = df.replace('European', 'white')
    df = df.replace('Hispanic and Portuguese', 'hispanic')
    df = df.replace('Likely African-American', 'black')
    return df

In [20]:
merged_file = replace_ethnicities(merged_file)
merged_file.head()

,LALVOTERID,Local_or_Municipal_2021_06_08,Local_or_Municipal_2021_04_20,Local_or_Municipal_2021_05_11,Local_or_Municipal_2019_08_13,Local_or_Municipal_2021_03_02,Local_or_Municipal_2017_03_07,General_2016_11_08,Local_or_Municipal_2017_05_09,Local_or_Municipal_2018_04_10,...,Voters_Gender,Voters_Age,Parties_Description,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate,County,CommercialData_Education,CommercialData_EstimatedHHIncomeAmount,FECDonors_NumberOfDonations,FECDonors_TotalDonationsAmount
0,LALCA453008306,None,None,None,None,None,None,None,None,None,...,F,26,Non-Partisan,black,04/01/2021,ALAMEDA,None,None,None,None
1,LALCA22129469,None,None,None,None,None,None,Y,None,None,...,F,47,Democratic,white,11/16/2021,ALAMEDA,HS Diploma - Extremely Likely,None,None,None
2,LALCA24729024,None,None,None,None,None,None,None,None,None,...,F,56,Democratic,white,02/28/2016,ALAMEDA,HS Diploma - Extremely Likely,None,None,None
3,LALCA22466723,None,None,None,None,None,None,None,None,None,...,F,38,Republican,white,11/01/2021,ALAMEDA,None,None,None,None
4,LALCA22466636,None,None,None,None,None,None,Y,None,None,...,M,63,Democratic,white,12/07/2021,ALAMEDA,None,None,None,None


In [21]:
GE_cols = [col for col in merged_file.columns if col.startswith('General')]
print(GE_cols)
LM_cols = [col for col in merged_file.columns if col.startswith('Local_or_Municipal')]
print(LM_cols)

['General_2016_11_08', 'General_2014_11_04', 'General_2020_11_03', 'General_2018_11_06']
['Local_or_Municipal_2021_06_08', 'Local_or_Municipal_2021_04_20', 'Local_or_Municipal_2021_05_11', 'Local_or_Municipal_2019_08_13', 'Local_or_Municipal_2021_03_02', 'Local_or_Municipal_2017_03_07', 'Local_or_Municipal_2017_05_09', 'Local_or_Municipal_2018_04_10', 'Local_or_Municipal_2016_04_12', 'Local_or_Municipal_2020_04_14', 'Local_or_Municipal_2019_04_16', 'Local_or_Municipal_2019_03_05', 'Local_or_Municipal_2019_08_27', 'Local_or_Municipal_2021_07_20', 'Local_or_Municipal_2017_06_06', 'Local_or_Municipal_2017_08_29', 'Local_or_Municipal_2021_06_01', 'Local_or_Municipal_2020_08_03', 'Local_or_Municipal_2019_06_04', 'Local_or_Municipal_2017_05_02', 'Local_or_Municipal_2018_07_24']


In [22]:
# fill NA values with "N" to make it easier to compare  with "Y"
merged_file[GE_cols+LM_cols] = merged_file[GE_cols+LM_cols].fillna('N')
merged_file.head()

,LALVOTERID,Local_or_Municipal_2021_06_08,Local_or_Municipal_2021_04_20,Local_or_Municipal_2021_05_11,Local_or_Municipal_2019_08_13,Local_or_Municipal_2021_03_02,Local_or_Municipal_2017_03_07,General_2016_11_08,Local_or_Municipal_2017_05_09,Local_or_Municipal_2018_04_10,...,Voters_Gender,Voters_Age,Parties_Description,EthnicGroups_EthnicGroup1Desc,Voters_OfficialRegDate,County,CommercialData_Education,CommercialData_EstimatedHHIncomeAmount,FECDonors_NumberOfDonations,FECDonors_TotalDonationsAmount
0,LALCA453008306,N,N,N,N,N,N,N,N,N,...,F,26,Non-Partisan,black,04/01/2021,ALAMEDA,None,None,None,None
1,LALCA22129469,N,N,N,N,N,N,Y,N,N,...,F,47,Democratic,white,11/16/2021,ALAMEDA,HS Diploma - Extremely Likely,None,None,None
2,LALCA24729024,N,N,N,N,N,N,N,N,N,...,F,56,Democratic,white,02/28/2016,ALAMEDA,HS Diploma - Extremely Likely,None,None,None
3,LALCA22466723,N,N,N,N,N,N,N,N,N,...,F,38,Republican,white,11/01/2021,ALAMEDA,None,None,None,None
4,LALCA22466636,N,N,N,N,N,N,Y,N,N,...,M,63,Democratic,white,12/07/2021,ALAMEDA,None,None,None,None


In [23]:
# We created the dataframe below in order to easily calculate perc_turnout when no one voted
list_ethnic_city = merged_file[['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc']].drop_duplicates()
list_ethnic_city_No = list_ethnic_city.copy()
list_ethnic_city_No['voted'] = 'N'
list_ethnic_city_Yes = list_ethnic_city.copy()
list_ethnic_city_Yes['voted'] = 'Y'
list_ethnic_city = pd.concat([list_ethnic_city_No, list_ethnic_city_Yes])

In [24]:
list_ethnic_city

,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,voted
0,Oakland,black,N
1,Oakland,white,N
2,San Leandro,white,N
3,Livermore,white,N
7,Oakland,asian,N
...,...,...,...
3802766,Santa Rosa,white,Y
3802769,Santa Rosa,hispanic,Y
3802772,Santa Rosa,asian,Y
3802827,Santa Rosa,black,Y


In [25]:
# we also need the total voters information per city and ethnicity
total_city_ethnic = merged_file.groupby(['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc']).size().reset_index()
total_city_ethnic.columns = ['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc', 'total_voters']
total_city_ethnic  = total_city_ethnic.merge(list_ethnic_city, on = ['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc'])

total_city_ethnic = replace_ethnicities(total_city_ethnic)
total_city_ethnic

,Residence_Addresses_City,EthnicGroups_EthnicGroup1Desc,total_voters,voted
0,Albany,asian,2405,N
1,Albany,asian,2405,Y
2,Albany,black,147,N
3,Albany,black,147,Y
4,Albany,hispanic,1035,N
...,...,...,...,...
315,Whittier,black,214,Y
316,Whittier,hispanic,76334,N
317,Whittier,hispanic,76334,Y
318,Whittier,white,26477,N


In [26]:
def calc_votes(df, col):
    voter_turnout_stats = df.groupby(['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc', col]).size().reset_index(name='voted_voters')

    # 'voted' is either 'Y' or 'N'
    voter_turnout_stats = voter_turnout_stats.rename(columns = {col: 'voted'})    

    voter_turnout_stats = total_city_ethnic.merge(voter_turnout_stats, 
                                                     how = 'left',
                                                     on = ['Residence_Addresses_City', 'EthnicGroups_EthnicGroup1Desc', 'voted']) 
    voter_turnout_stats['perc_turnout'] = voter_turnout_stats['voted_voters']/voter_turnout_stats['total_voters']

    voter_turnout_stats['elec_date'] = col[len(col)-10:]
    voter_turnout_stats['elec_year'] = col[len(col)-10:len(col)-6]
    voter_turnout_stats['elec_type'] = col[:len(col)-11]

    voter_turnout_stats[['voted_voters', 'perc_turnout']] = voter_turnout_stats[['voted_voters', 'perc_turnout']].fillna(0)
    voter_turnout_stats = voter_turnout_stats[voter_turnout_stats['voted'] == 'Y']    
    pivot_df = voter_turnout_stats.pivot(index = ['elec_type','elec_year', 'elec_date', 'Residence_Addresses_City'],
                                    columns='EthnicGroups_EthnicGroup1Desc', 
                                    values=['total_voters', 'voted_voters', 'perc_turnout']).reset_index()
    pivot_df.columns = pivot_df.columns.map('_'.join)
    pivot_df = pivot_df.rename(columns = {'elec_type_':'elec_type', 'elec_year_':'elec_year', 'elec_date_':'elec_date', 'Residence_Addresses_City_':'Residence_Addresses_City'})

    del voter_turnout_stats
    gc.collect()
    return pivot_df

elec_date_cols = GE_cols+LM_cols

for i in range(len(elec_date_cols)):
    col = elec_date_cols[i]
    pivot_df = calc_votes(merged_file, col)    
    # stack all types of election into one dataframe 
    if i == 0:
        voter_turnout_merge_ethnicity = pivot_df.copy() 
    else:
        voter_turnout_merge_ethnicity = pd.concat([voter_turnout_merge_ethnicity, pivot_df])


In [27]:
voter_turnout_merge_ethnicity.head()

,elec_type,elec_year,elec_date,Residence_Addresses_City,total_voters_asian,total_voters_black,total_voters_hispanic,total_voters_white,voted_voters_asian,voted_voters_black,voted_voters_hispanic,voted_voters_white,perc_turnout_asian,perc_turnout_black,perc_turnout_hispanic,perc_turnout_white
0,General,2016,2016_11_08,Albany,2405.0,147.0,1035.0,6169.0,1396.0,88.0,668.0,4659.0,0.580457,0.598639,0.645411,0.755228
1,General,2016,2016_11_08,Alhambra,17451.0,191.0,16596.0,7359.0,7041.0,90.0,9717.0,4574.0,0.403473,0.471204,0.585503,0.621552
2,General,2016,2016_11_08,Anaheim,26340.0,1211.0,70052.0,54644.0,12245.0,609.0,34883.0,35829.0,0.464882,0.502890,0.497959,0.655680
3,General,2016,2016_11_08,Bellflower,2153.0,3614.0,19899.0,10792.0,840.0,2043.0,9474.0,6092.0,0.390153,0.565302,0.476104,0.564492
4,General,2016,2016_11_08,Berkeley,8549.0,5942.0,6388.0,39425.0,4602.0,3770.0,3636.0,27737.0,0.538309,0.634467,0.569192,0.703538


In [28]:
print(voter_turnout_merge_ethnicity.shape)
# remove rows where election dates are not associated with city
voter_turnout_merge_ethnicity = GE_LM_dates_df.merge(voter_turnout_merge_ethnicity, 
                                how = 'left',
                                on = ['elec_type', 'elec_year', 'elec_date', 'Residence_Addresses_City'])
print(voter_turnout_merge_ethnicity.shape)

(1000, 16)
(320, 16)


In [29]:
#should be empty dataframe because of the way we have filitered the dataframe

no_voter_turnout = voter_turnout_merge_ethnicity[(voter_turnout_merge_ethnicity['perc_turnout_asian'] == 0) &
                                       (voter_turnout_merge_ethnicity['perc_turnout_black'] == 0) &
                                       (voter_turnout_merge_ethnicity['perc_turnout_hispanic'] == 0) &
                                       (voter_turnout_merge_ethnicity['perc_turnout_white'] == 0)]

no_voter_turnout.head()

,Residence_Addresses_City,elec_date,elec_year,elec_type,total_voters_asian,total_voters_black,total_voters_hispanic,total_voters_white,voted_voters_asian,voted_voters_black,voted_voters_hispanic,voted_voters_white,perc_turnout_asian,perc_turnout_black,perc_turnout_hispanic,perc_turnout_white


#  3.2.  Calculate average donation 

In [30]:
def calc_donation(df):
    donations_df = df[['Residence_Addresses_City', 'FECDonors_TotalDonationsAmount', 'FECDonors_NumberOfDonations']
                  + elec_date_cols]
    melt_donations_df = donations_df.melt(id_vars=['Residence_Addresses_City', 'FECDonors_TotalDonationsAmount', 'FECDonors_NumberOfDonations'], 
              value_vars=elec_date_cols,
              var_name='elec_type_date',
              value_name='voted')
    melt_donations_df = melt_donations_df[melt_donations_df['voted'] == 'Y']

    melt_donations_df = melt_donations_df.astype({'FECDonors_TotalDonationsAmount': float, 'FECDonors_NumberOfDonations': float})                        
    melt_donations_df = melt_donations_df.groupby(['Residence_Addresses_City', 'elec_type_date']).agg({'FECDonors_TotalDonationsAmount':'sum','FECDonors_NumberOfDonations':'sum'}).reset_index()    
    melt_donations_df['mean_donation'] = melt_donations_df['FECDonors_TotalDonationsAmount']/melt_donations_df['FECDonors_NumberOfDonations']
    melt_donations_df['elec_date'] = melt_donations_df['elec_type_date'].str[-10:]
    melt_donations_df['elec_year'] = melt_donations_df['elec_type_date'].str[-10:-6]
    melt_donations_df['elec_type'] = melt_donations_df['elec_type_date'].str[:-11]
    melt_donations_df = melt_donations_df.drop(columns = 'elec_type_date').reset_index(drop=True)
    
    return melt_donations_df

avg_donations = calc_donation(merged_file)
print(avg_donations.shape)
avg_donations.head()

(537, 7)


,Residence_Addresses_City,FECDonors_TotalDonationsAmount,FECDonors_NumberOfDonations,mean_donation,elec_date,elec_year,elec_type
0,Albany,2433223.0,24639.0,98.754941,2014_11_04,2014,General
1,Albany,2803209.0,27551.0,101.746180,2016_11_08,2016,General
2,Albany,2753784.0,26649.0,103.335360,2018_11_06,2018,General
3,Albany,2839804.0,28062.0,101.197491,2020_11_03,2020,General
4,Albany,2840.0,43.0,66.046512,2017_03_07,2017,Local_or_Municipal


In [31]:
#don't need to do this for other dataframes, using inner join will ensure only necessary combinations of city and election dates are present
# avg_donations = GE_LM_dates_df.merge(avg_donations, 
#                                      how = 'left',
#                                      on = ['elec_type', 'elec_year', 'elec_date', 'Residence_Addresses_City'])
# print(avg_donations.shape)

In [32]:
# Meger 3.1 and 3.2
voter_turnout_merge = voter_turnout_merge_ethnicity.merge(avg_donations, 
                                                          how = 'inner',
                                                          on = ['elec_type', 'elec_year', 'elec_date', 'Residence_Addresses_City'])

voter_turnout_merge.head()

,Residence_Addresses_City,elec_date,elec_year,elec_type,total_voters_asian,total_voters_black,total_voters_hispanic,total_voters_white,voted_voters_asian,voted_voters_black,voted_voters_hispanic,voted_voters_white,perc_turnout_asian,perc_turnout_black,perc_turnout_hispanic,perc_turnout_white,FECDonors_TotalDonationsAmount,FECDonors_NumberOfDonations,mean_donation
0,Oakland,2020_11_03,2020,General,30600.0,61476.0,37174.0,83122.0,23041.0,45891.0,26954.0,69989.0,0.752974,0.746486,0.725077,0.842003,42110405.0,386556.0,108.937398
1,Oakland,2018_11_06,2018,General,30600.0,61476.0,37174.0,83122.0,14972.0,35012.0,17857.0,57872.0,0.489281,0.569523,0.480363,0.696230,40806061.0,378375.0,107.845553
2,Oakland,2016_11_08,2016,General,30600.0,61476.0,37174.0,83122.0,16057.0,37256.0,19792.0,57968.0,0.524739,0.606025,0.532415,0.697385,40282807.0,374104.0,107.678098
3,Oakland,2014_11_04,2014,General,30600.0,61476.0,37174.0,83122.0,8145.0,21265.0,8235.0,35411.0,0.266176,0.345907,0.221526,0.426012,35190358.0,332135.0,105.951971
4,San Leandro,2020_11_03,2020,General,12705.0,5596.0,16028.0,17780.0,9229.0,4299.0,11984.0,14638.0,0.726407,0.768227,0.747692,0.823285,1835380.0,30062.0,61.053157


#  3.3.  Calculate average income 

#  3.4.  Calculate Voter turnout or percentage of Education?

#  3.4.  Calculate voter turnout per age group

# Save the merged aggregations 

In [33]:
# voter_turnout_merged.to_csv(f'{filepath}voter_turnout_merged.csv', index=False)

In [34]:
# del voter_turnout_merge
# gc.collect()

In [35]:
end_time = time.time()
print("Time take to run this notebook in seconds: ", end_time - start_time)

Time take to run this notebook in seconds:  183.3288700580597
